## Divvy Bikes

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.api import VAR



## Divvy Bikes VAR

In [9]:
mdata = pd.read_csv('dataset/DivvyBikes/df2021-8-9-10.csv')
hourly_checkout = [ic for ic in mdata.columns if "_checkout" in ic]
hourly_checkout.insert(0,"checkouthour") ## this will be replaced with date at the end
hourly_checkout=hourly_checkout[:-1]
mdata = mdata[hourly_checkout[:201]]
mdata.rename(columns={"checkouthour":"date"},inplace=True)
mdata.set_index('date', inplace = True)
mdata.to_csv("dataset/DivvyBikes/df2021-8-9-10_VAR.csv")

## Kaggle VAR

In [54]:
kaggledata = pd.read_csv("dataset/KaggleStoreSales/train.csv")
regions = list(kaggledata)[1:]
kaggledata['category'] = kaggledata['country'] + '_' + kaggledata['store'] + '_' + kaggledata['product']

df_tmp = kaggledata.set_index("category")
df_new = pd.DataFrame(df_tmp[df_tmp.index == "Spain_Kagglazon_Using LLMs to Write Better"].date)
df_new = df_new.reset_index().drop("category",axis=1)

for idx in df_tmp.index.unique():
    df_part = df_tmp[df_tmp.index == idx]
    df_new[idx] = df_part.num_sold.values



## logic for the diff 
#for icol in list(df_new)[1:]:
#    df_new[icol] = df_new[icol].diff(1).fillna(0)
    
#df_new
df_new.set_index("date",inplace=True)
df_new.to_csv('dataset/KaggleStoreSales/train_VAR.csv')

In [56]:
!head dataset/KaggleStoreSales/train_VAR.csv

date,Argentina_Kaggle Learn_Using LLMs to Improve Your Coding,Argentina_Kaggle Learn_Using LLMs to Train More LLMs,Argentina_Kaggle Learn_Using LLMs to Win Friends and Influence People,Argentina_Kaggle Learn_Using LLMs to Win More Kaggle Competitions,Argentina_Kaggle Learn_Using LLMs to Write Better,Argentina_Kaggle Store_Using LLMs to Improve Your Coding,Argentina_Kaggle Store_Using LLMs to Train More LLMs,Argentina_Kaggle Store_Using LLMs to Win Friends and Influence People,Argentina_Kaggle Store_Using LLMs to Win More Kaggle Competitions,Argentina_Kaggle Store_Using LLMs to Write Better,Argentina_Kagglazon_Using LLMs to Improve Your Coding,Argentina_Kagglazon_Using LLMs to Train More LLMs,Argentina_Kagglazon_Using LLMs to Win Friends and Influence People,Argentina_Kagglazon_Using LLMs to Win More Kaggle Competitions,Argentina_Kagglazon_Using LLMs to Write Better,Canada_Kaggle Learn_Using LLMs to Improve Your Coding,Canada_Kaggle Learn_Using LLMs to Train More LLMs,Canada_Kaggle Lear

In [ ]:
regions = list(df_new)[1:]
df_global = pd.DataFrame()
for col in regions:
    #print (col)
    aux = df_new[["date", col]].copy(deep=True)  # select column associated with region
    aux = aux.iloc[:, :].copy(deep=True)  # selects data up to 26301 row (2004 to 2007 time stamps)
    aux = aux.rename(columns={col: "num_sold_diff"})  # rename column of data to 'y' which is compatible with Neural Prophet
    aux["category"] = col
    df_global = pd.concat((df_global, aux))
df = df_global.merge(df,on=["date","category"],how='inner').drop('category',axis=1)